In [269]:
!pip install py_arkworks_bls12381

from py_arkworks_bls12381 import G1Point, G2Point, GT, Scalar
import random
from random import randint
from numpy.polynomial import polynomial as P
import numpy as np

In [270]:
# Tools

# Check if the polynomial's coefs are integer
def assert_integer_polynomial(f):
    for i in f.coef:
        assert int(i) == i

In [271]:
# Initialize the Elliptic Curve
g = G1Point()
h = G2Point()
gt_gen = GT()

g_identity = G1Point.identity()
h_identity = G2Point.identity()

In [272]:
class Trusted_Setup:
    def __init__(self, d):
        self.d = d

    # input d, the maximum degree of a polynomial we will commit to
    # returns (ck, vk)
    # ck is the commitment key which is the list: ([1]_1, [tau]_1, ..., [tau ^ d]_1)
    # vk is the verification key which is a pair: ([1]_2, [tau]_2)
    def generate_public_parameters(self):
        keys = {}
        keys["ck"] =[];
        keys["vk"] =[];

        tau = Scalar(randint(0, 2^255 - 1))

        coef = Scalar(1)
        for _ in range(self.d):
            keys["ck"].append(g * coef)
            coef *= tau

        keys["vk"].append(h)
        keys["vk"].append(h * tau)

        return keys

In [273]:
class Prover:
    def __init__(self, ck, f):
        self.ck = ck
        self.f = f

    # input f is the polynomial we are commiting to
    # input ck is a list. The commitment key generated by the trusted setup
    # returns c_f, the commited point on G1
    def commit(self):
        assert_integer_polynomial(self.f)

        c_f = g_identity
        for i in range(len(self.f.coef)):
            f_i = int(self.f.coef[i])
            c_f += self.ck[i] * Scalar(f_i)

        return c_f

    # input s is the point on G1
    def eval(self, s):
        z = self.f(s)

        numerator_poly = self.f.copy()
        numerator_poly.coef[0] -= z # If the numbers are too large, we fall into an overflow here

        denominator_poly = P.Polynomial([-s, 1])
        division_result = P.polydiv(numerator_poly.coef, denominator_poly.coef)
        q = P.Polynomial(division_result[0])

        c_q = g_identity
        for i in range(len(q)):
            q_i = int(q.coef[i])
            c_q += self.ck[i] * Scalar(q_i)

        return c_q

In [274]:
class Verifier:
    def __init__(self, vk):
        self.vk = vk

    # c_f: commitment to the polynomial
    # evaluation of f on s is equal to z: f(s) = z
    # c_q commitment to the derived polynomial
    def verify(self, c_f, s, z, c_q):
        assert(self.vk[0] == h)

        lhs = GT.pairing(c_f - g * Scalar(z), self.vk[0])
        rhs = GT.pairing(c_q, self.vk[1] - h * Scalar(s))

        return lhs == rhs

In [275]:
# Test

max_d = 100
trusted_setup = Trusted_Setup(max_d)
pp = trusted_setup.generate_public_parameters()
ck = pp["ck"]
vk = pp["vk"]

def manual_polynomial_correct_commitment():
    f = P.Polynomial([1, 2, 3])
    prover = Prover(ck, f)
    verifier = Verifier(vk)

    c_f = prover.commit()
    s = 2
    z = int(f(s))
    c_q = prover.eval(s)

    return verifier.verify(c_f, s, z, c_q)

def small_random_polynomial_correct_commitment():
    f_deg = 10
    max_coef = 10

    f = P.Polynomial([np.float128(randint(0, max_coef)) for _ in range(f_deg)])

    prover = Prover(ck, f)
    verifier = Verifier(vk)

    c_f = prover.commit()
    s = randint(0, 100)
    z = int(f(s))
    c_q = prover.eval(s)

    return verifier.verify(c_f, s, z, c_q)

def small_random_polynomial_wrong_evaluation_commitment():
    f_deg = 7
    max_coef = 7

    f = P.Polynomial([np.float128(randint(0, max_coef)) for _ in range(f_deg)])

    prover = Prover(ck, f)
    verifier = Verifier(vk)

    c_f = prover.commit()
    s = randint(0, 100)
    z = int(f(s))
    wrong_z = z + 1
    c_q = prover.eval(s)

    return verifier.verify(c_f, s, wrong_z, c_q)

assert(manual_polynomial_correct_commitment() == True)
assert(small_random_polynomial_wrong_evaluation_commitment() == False)
assert(small_random_polynomial_correct_commitment() == True)